<a href="https://colab.research.google.com/github/punkmic/pytorch_neural_network/blob/master/convolution_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Pytorch - Convolution Neural Network**

In [10]:
import numpy as np
try:
  import torch
  from torch import nn, optim
  import torch.nn.functional as F
  import torchvision
  from torchvision import datasets, transforms
except:
  !pip install torch

In [11]:
def train(model, train_loader, cost, optimizer, epoch):
  """
    Train a covolution neural network model.
    
    Parameters:
    model (nn.Module): The model to be trained.
    train_loader (DataLoader): The training data loader.
    cost (function): The cost function used to evaluate the model's performance during training.
    optimizer (Optimizer): The optimization algorithm used to update the model's parameters.
    epoch (int): The number of iterations over the entire training data.
    
    Returns:
    None
  """
  # Loop through each epoch
  for e in range(epoch):
    # Initialize running loss to 0
    running_loss = 0
    # Initialize the number of correct predictions to 0
    correct = 0
    # Loop through each batch in the train_loader
    for data, target in train_loader:
      # Zero the gradients of the model parameters
      optimizer.zero_grad()
      # Get the predictions from the model
      pred = model(data)
      # Calculate the loss between the predictions and the target
      loss = cost(pred, target)
      # Add the loss to the running total of the loss
      running_loss += loss
      # Compute the gradients of the loss with respect to the model parameters
      loss.backward()
      # Update the model parameters using the optimizer
      optimizer.step()
      # Find the index of the maximum prediction for each sample in the batch
      pred = pred.argmax(dim=1, keepdim=True)
      # Increase the number of correct predictions
      correct += pred.eq(target.view_as(pred)).sum().item()
    # Print the average loss and accuracy over the entire training dataset at the end of each epoch
    print(f"Epoch {e}: Loss {running_loss/len(train_loader.dataset)}, Accuracy {100*(correct/len(train_loader.dataset))}%")

In [12]:
def test(model, test_loader):
    """
    Test a given model on the test dataset.
    
    Parameters:
        model (nn.Module): The model to be tested.
        test_loader (torch.utils.data.DataLoader): The dataloader for the test data.
        
    Returns:
        None
        
    """
    model.eval()
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            # Calculate the predicted outputs
            output = model(data)
            # Find the class with maximum probability
            pred = output.argmax(dim=1, keepdim=True)
            # Count the number of correctly classified samples
            correct += pred.eq(target.view_as(pred)).sum().item()

    # Print the accuracy of the model on the test dataset
    print(f'Test set: Accuracy: {correct}/{len(test_loader.dataset)} = {100*(correct/len(test_loader.dataset))}%)')

In [13]:
class Model(nn.Module):
    """
    A convolutional neural network model for image classification.
    
    The model consists of two convolutional layers, followed by three fully-connected layers. 
    The activation function used is ReLU.
    
    """
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)  # First convolutional layer, 3 input channels, 8 output channels, 5x5 kernel
        self.pool = nn.MaxPool2d(2, 2)  # Max pooling layer with 2x2 window and stride 2
        self.conv2 = nn.Conv2d(6, 16, 5)  # Second convolutional layer, 8 input channels, 24 output channels, 5x5 kernel
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # First fully-connected layer, 120 output units
        self.fc2 = nn.Linear(120, 84)  # Second fully-connected layer, 84 output units
        self.fc3 = nn.Linear(84, 42)  # Third fully-connected layer, 42 output units
        self.fc4 = nn.Linear(42, 10)  # Output layer, 10 output units

    def forward(self, x):
        """
        Define the forward pass of the model.
        
        Parameters:
            x (torch.Tensor): Input tensor of shape (batch_size, 3, 32, 32)
            
        Returns:
            torch.Tensor: Output tensor of shape (batch_size, 10)
        """
        x = self.pool(F.relu(self.conv1(x)))  # Apply first convolutional layer and max pooling
        x = self.pool(F.relu(self.conv2(x)))  # Apply second convolutional layer and max pooling
        x = torch.flatten(x, 1)  # Flatten the tensor to 2D
        x = F.relu(self.fc1(x))  # Apply first fully-connected layer with ReLU activation
        x = F.relu(self.fc2(x))  # Apply second fully-connected layer with ReLU activation
        x = F.relu(self.fc3(x))  # Apply third fully-connected layer with ReLU activation
        x = self.fc4(x)  # Apply output layer
        return x


In [14]:
# Setting batch size and number of epochs
batch_size = 32
epoch = 20

# Define transforms for training data
training_transform = transforms.Compose([
    # Apply random horizontal flip with probability 0.5
    transforms.RandomHorizontalFlip(p=0.5),
    # Convert the data to tensor
    transforms.ToTensor(),
    # Normalize the data with mean and standard deviation
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Define transforms for test data
testing_transform = transforms.Compose([
    # Convert the data to tensor
    transforms.ToTensor(),
    # Normalize the data with mean and standard deviation
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Load CIFAR10 training dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
        download=True, transform=training_transform)

# Create a dataloader for the training dataset
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
        shuffle=True)

# Load CIFAR10 test dataset
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
        download=True, transform=testing_transform)

# Create a dataloader for the test dataset
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
        shuffle=False)

# Create a model instance
model=Model()

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define the optimizer
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Train the model
train(model, trainloader, criterion, optimizer, epoch)

# Test the model on the test dataset
test(model, testloader)

Files already downloaded and verified
Files already downloaded and verified
Epoch 0: Loss 0.07197240740060806, Accuracy 10.042%
Epoch 1: Loss 0.06932945549488068, Accuracy 18.358%
Epoch 2: Loss 0.060444436967372894, Accuracy 27.548000000000002%
Epoch 3: Loss 0.05445527657866478, Accuracy 35.066%
Epoch 4: Loss 0.04930206015706062, Accuracy 42.13%
Epoch 5: Loss 0.045639656484127045, Accuracy 46.214%
Epoch 6: Loss 0.0429612398147583, Accuracy 49.71%
Epoch 7: Loss 0.04065779224038124, Accuracy 52.736000000000004%
Epoch 8: Loss 0.0387149453163147, Accuracy 55.406%
Epoch 9: Loss 0.03718198463320732, Accuracy 57.611999999999995%
Epoch 10: Loss 0.035558685660362244, Accuracy 59.53000000000001%
Epoch 11: Loss 0.03423401340842247, Accuracy 61.18%
Epoch 12: Loss 0.033107057213783264, Accuracy 62.327999999999996%
Epoch 13: Loss 0.032091833651065826, Accuracy 63.592000000000006%
Epoch 14: Loss 0.03118574060499668, Accuracy 64.834%
Epoch 15: Loss 0.03036535158753395, Accuracy 65.648%
Epoch 16: Loss 